<a href="https://colab.research.google.com/github/shiv6789/Solidity/blob/main/Blockchain(Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import datetime 
import hashlib
import json
from flask import Flask,jsonify

#part 1 :Building a Blockchain
class Blockchain:

  def __init__(self):
    self.chain=[]#formation of blockchain in init method
    self.createBlock(proof=1,previousHash='0')#genesisi block

  def createBlock(self,proof,previousHash):
    #following code of block is for dictionary wher there is key and values.
    #dictionary is similiar to hash map in javva.
    block={"index": len(self.chain)+1,
           "timestamp": str(datetime.datetime.now()),
           "proof": proof,
           "previous hash": previousHash
           }
    self.chain.append(block)#adding block ti block chain.
    return block 

  def getPreviousBlock(self):
     return self.chain[-1]#-1] means the last element in a sequence

  def proofOfWork(self,previousProof):
    newProof=1
    checkProof=False
    while checkProof is False:
      hashOperations=hashlib.sha256(str(newProof**2-previousProof**2).encode).hexdigest()#hexdigest:it converst a encoded data to hex decimal format.
      if hashOperations[:4]=="0000":#to check that hash return must have first four charcter of strring must be zero.
        checkProof=True
      else:
        newProof+=1;
    return newProof

  def hash(self,block):
      encodedBlock=json.dumps(block,sort_keys=True).encode()#converts python dicttionarry to json string
      return hashlib.sha256(encodedBlock).hexdigest()

  def isChainValid(self,chain):
    previousBlock=chain[0]#chain[0] is genesisi block.
    blockIndex=1;
    while blockIndex < len(chain):
      block=chain[blockIndex]
      if block["previous hash"]!=self.hash(previousBlock):#checking whether previous hash is matching or not
        return False
      previousProof=previousBlock["proof"]
      proof=block["proof"]
      hashOperations=hashlib.sha256(str(proof**2-previousProof**2).encode()).hexidigest()
      if(hashOpeartions[:4]!='0000'):#checking wheteher its following proof of work condition.
       return False
      
      previousBlock=block
      blockIndex+=1;#iterating complete blockchain one by one for validations.
    
    return True

#part 2 :Mining A bLockchain

#creating a web app
app=Flask(__name__)

#creating a blockchain
blockchain=Blockchain()

#mining a block
@app.route('/mineBlock',methods=['GET'])

def mineBlock():
  previousBlock=blockchain.getPreviousBlock()
  previousProof=previousBlock["proof"]
  previousHash=blockchain.hash(previousBlock)
  proof=blockchain.proofOfWork(previousProof)
  block=blockchain.createBlock(proof,previousHash)
  response={"message":"Congratulations,You Have Succesfully Mined A Block",
            "index":block["index"],
            "Timestamp":block["timestamp"],
            "proof":block["proof"],
            "previousHash":block["previousHash"]}

  return jsonify(response),200 #sonify() is a helper method provided by Flask to properly return JSON data. jsonify() returns a Response object with the application/json mimetype set, whereas json.dumps() simply returns a string of JSON data. This could lead to unintended results.